# Generando EVI_RECLASS

In [ ]:
import os, rasterio, subprocess

escena = r'Y:\Proyectos\marina_valdecanas\capas\teledeteccion\Landsat\pro\20161004l8oli202_32'

#for pro in os.listdir(path_pro):
    
#escena = os.path.join(path_pro, pro)

for i in os.listdir(escena):

   

    if i.endswith('evi_rec.img'):
        evi = os.path.join(escena, i)

    elif i.endswith('water_reclass_3.img'):
            watermask = os.path.join(escena, i)

with rasterio.open(evi) as src_evi:
    EVIREC = src_evi.read()
with rasterio.open(watermask) as src_msk:
    WATER = src_msk.read()

EVIREC[WATER != 1] = 0

outfile = os.path.join(escena, os.path.split(escena)[1] + '_evi_reclass.img')

profile = src_evi.meta
#profile.update(dtype=rasterio.float32)

with rasterio.open(outfile, 'w', **profile) as dst:
    dst.write(EVIREC.astype(rasterio.float32))

print(escena, 'realizada')



# Generando EVI_RECLASS a 0.05

In [ ]:
import os, rasterio, subprocess
import numpy as np

path_pro = r'Y:\Proyectos\marina_valdecanas\capas\teledeteccion\Landsat\pro'

for pro in os.listdir(path_pro):
       
    escena = os.path.join(path_pro, pro)

    if os.path.isdir(escena):

        for i in os.listdir(escena):

            try:

                if i.endswith('_evi_reclass.img'):
                    evi = os.path.join(escena, i)


                    #print(watermask,'\n', evi)
                    #Ya tenemos la variables en cada escena, ahora hacemos la reclasificación para que lo que no sea agua valga 0, y 
                    #no alterar la suma metiendo valores altos de pastizales que pudiera haber en las orillas

                    with rasterio.open(evi) as src_evi:
                        EVIREC = src_evi.read()

                    #Ahora mismo hacemos un reclassify entre lo que es agua y lo que no, para poder calcualr después frecuencias
                    EVIRECLASS = np.where(EVIREC >= 0.05, 1, 0)

                    #outfile = os.path.join(escena, os.path.split(escena)[1] + '_binwaterB.img')
                    outfile = os.path.join(escena, os.path.split(escena)[1] + '_evi_reclass_05.img')
                    
                    profile = src_evi.meta
                    #profile.update(dtype=rasterio.float32)

                    with rasterio.open(outfile, 'w', **profile) as dst:
                        dst.write(EVIRECLASS.astype(rasterio.float32))

                    print(escena, 'realizada')

            except Exception as e:
                print(escena, e)
                continue

# Ahora tenemos que crear para cada periodo, un raster con el número de veces que el pixel es valido y la suma de los días que ha estado por encima de 0.05

In [ ]:
import os
import rasterio
from rasterio import plot
import matplotlib.pyplot as plt

rutapro = r'Y:\Proyectos\marina_valdecanas\capas\teledeteccion\Landsat\pro'
rutasave = r'C:\Users\equipo\Desktop\Valdecañas\EVI\raster'

evisPres = []
evisDur = []
evisPost = []

WS_Pres = []
WS_Dur = []
WS_Post = []

In [ ]:
for i in os.listdir(rutapro):
    if os.path.isdir(os.path.join(rutapro, i)) and not 'EVI' in i:
        if int(i[:6]) < 200707:
            escena = os.path.join(rutapro, i)
            for f in os.listdir(escena):
                if f.endswith('_evi_reclass_05.img'):
                    evisPres.append(os.path.join(escena, f))
                elif f.endswith('_binwater.img'):
                    WS_Pres.append(os.path.join(escena, f))
        elif int(i[:6]) > 201107:
            escena = os.path.join(rutapro, i)
            for f in os.listdir(escena):
                if f.endswith('_evi_reclass_05.img'):
                    evisPost.append(os.path.join(escena, f))
                elif f.endswith('_binwater.img'):
                    WS_Post.append(os.path.join(escena, f))
        else:
            escena = os.path.join(rutapro, i)
            for f in os.listdir(escena):
                if f.endswith('_evi_reclass_05.img'):
                    evisDur.append(os.path.join(escena, f))
                elif f.endswith('_binwater.img'):
                    WS_Dur.append(os.path.join(escena, f))

In [ ]:
print(len(evisPres), len(evisDur), len(evisPost))

# Hacemos dos listas con los 3 periodos y hacemos un stack para cada periodo y variable (EVIs y Waterbin)

In [ ]:
#Listas con los periodos
evis = [sorted(evisPres), sorted(evisDur), sorted(evisPost)]
waters = [sorted(WS_Pres), sorted(WS_Dur), sorted(WS_Post)]

#Listas con los nombres de los stacks que se generaraán
evisout = [os.path.join(rutasave, 'evis_pres_stack.img'), 
           os.path.join(rutasave, 'evis_dur_stack.img'), 
           os.path.join(rutasave, 'evis_post_stack.img')]
watersout = [os.path.join(rutasave, 'waterbin_pres_stack.img'), 
           os.path.join(rutasave, 'waterbin_dur_stack.img'), 
           os.path.join(rutasave, 'waterbin_post_stack.img')]

#Lista con las dos variables
paramsIn = [evis, waters]
paramsOut = [evisout, watersout]

In [ ]:
d = {'pres':[], 'dur':[], 'post':[]}

for i in os.listdir(rutasave):
    if i.endswith('.img'):
        param = i.split('_')[1]
        if i.startswith('evis'):
            d[param].append(os.path.join(rutasave, i))
        elif i.startswith('water'):
            d[param].append(os.path.join(rutasave, i))
            

In [ ]:
d

In [ ]:
#Esto se hace para cada uno de los periodos de forma automática
for nn, ee in enumerate(paramsIn):
    
    for n, e in enumerate(ee):
    
    
        outfile = paramsOut[nn][n]
        print(n, e[0])
        print('Outfile: ', outfile)

        # Read metadata of first file
        with rasterio.open(e[0]) as src0:
            meta = src0.meta

        # Update meta to reflect the number of layers
        meta.update(count = len(e))

        # Read each layer and write it to stack
        with rasterio.open(outfile, 'w', **meta) as dst:
            for id, layer in enumerate(e):
                with rasterio.open(layer) as src1:
                    dst.write_band(id + 1, src1.read(1))
        

# Hacemos la suma con de los stacks

In [ ]:
import numpy as np

for k, v in d.items():
    #V[0] es EVI y v[1] es water
    outfile = os.path.join(rutasave, 'sumNan_water_' + k + '.img')
    with rasterio.open(v[1]) as src0:
        
        EVI = src0.read()
        EVISUM = np.nansum(EVI, axis=0)

        meta = src0.meta
        meta.update(count=1)
        meta.update(dtype=rasterio.int16)      

        # Read each layer and write it to stack
        with rasterio.open(outfile, 'w', **meta) as dst:
            dst.write_band(1, EVISUM.astype(rasterio.int16))

# Tenemos un diccionario con los evis y su acumulado de agua, ahora sacamos la frecuencia

In [ ]:
dsum = {'pres':[], 'dur':[], 'post':[]}

for i in os.listdir(rutasave):
    
    if i.endswith('.img') and i.startswith('sum_'):
        
        param = i.split('_')[2][:-4]
        if 'evi' in i:
            dsum[param].append(os.path.join(rutasave, i))
        elif 'water' in i:
            dsum[param].append(os.path.join(rutasave, i))
            

In [ ]:
dsum

In [ ]:
import os, rasterio, subprocess


for k, v in dsum.items():
    
    outfile = os.path.join(rutasave, 'fre_' + k + '.img')
    
    with rasterio.open(v[0]) as sumaevi:
        SUMAEVI = sumaevi.read()
    
    with rasterio.open(v[1]) as sumawater:
        SUMAWATER = sumawater.read()

    FRECUENCY = (SUMAEVI / SUMAWATER) * 100

    profile = sumaevi.meta
    profile.update(dtype=rasterio.float32)

    with rasterio.open(outfile, 'w', **profile) as dst:
        dst.write(FRECUENCY.astype(rasterio.float32))

    print('realizado')

# Comprobado que los calculos son correctos (cotejados con np.nanmean) vamos a pasar los raster de frecuencias a byte (valores de 0 a 100 en enteros, a que no se precisan decimales) y los rasters de Media de EVI a Enteros de 0 a 10.000

Pasasmos los rasters de Freq a Byte con GDAL_TRANSLATE, se hace redondeando los valores al entero más cercano

In [ ]:
!gdal_translate -ot Byte C:\Users\equipo\Desktop\Valdecañas\EVI\raster\fre_pres.img C:\Users\equipo\Desktop\Valdecañas\EVI\raster\byte\freq_presB.img

Ahora vamos a pasar las medias a la escala de MODIS (-10000 - 10000)

In [ ]:
import rasterio

path = r'Y:\Proyectos\marina_valdecanas\capas\teledeteccion\Landsat\pro\Analisis_EVI\3_periodos\raster'

preMean = os.path.join(path, 'eviPre_mean.img')
durMean = os.path.join(path, 'eviDur_mean.img')
PostMean = os.path.join(path, 'eviPost_mean.img')

In [ ]:
outfile = r'C:\Users\equipo\Desktop\Valdecañas\EVI\raster\byte\PostMean10K.img'

with rasterio.open(PostMean) as src:
    
    POSTMEAN = src.read()

    POSTMEAN10K = POSTMEAN * 10000

    profile = src.meta
    profile.update(dtype=rasterio.int32)

with rasterio.open(outfile, 'w', **profile) as dst:
    dst.write(POSTMEAN10K.astype(rasterio.int32))

print('realizado')